# Algoritmo Genético para o Problema da Mochila
Autor: Prof. Maicon Melo Alves, Dsc. (YDUQS - Wyden)

Esse algoritmo foi criado com a intenção de fornecer um conteúdo prático para o estudo de um algoritmo genético. É, portanto, uma versão eminentemente didática. 

Esse código pode ser aberto no Jupyter Notebook ou no Google Colab. 

As instâncias para o problema da mochila foram obtidas desse site: http://artemisa.unicauca.edu.co/~johnyortega/instances_01_KP/

### Importação de bibliotecas

In [ ]:
import random
import numpy as np
from operator import itemgetter

### Definição de funções utilizadas pelo programa

Função para realizar a leitura da instâcia do problema. 

In [ ]:
def ler_instancia(arquivo):
    with open(arquivo) as arq:
        linhas = arq.readlines()
        instancia = {}
        instancia['numero_itens'] = int(linhas[0].split()[0])
        instancia['capacidade'] = int(linhas[0].split()[1])
        instancia['itens'] = []
        
        for linha in linhas[1:instancia['numero_itens']+1]:
            item = {}
            item['valor'] = int(linha.split()[0])
            item['peso'] = int(linha.split()[1])
            instancia['itens'].append(item)
        
    return instancia

Função para realizar a etapa de selação dos indivíduos mais aptos da população

In [ ]:
def selecionar_individuos(populacao):
    populacao_ordenada = sorted(populacao, key=itemgetter('aptidao'), reverse=True)
    
    return populacao_ordenada[:SELECIONADOS]


Função para criar um indivíduo e calcular o seu grau de aptidão

In [ ]:
def criar_individuo(representacao, instancia):
    individuo = {}
    individuo['representacao'] = representacao
    individuo['aptidao'] = grau_aptidao(individuo, instancia)
    
    return individuo


Função para realizar a etapa de reprodução dos indivíduos mais aptos. 

In [ ]:
def reproduzir(populacao, ponto_corte, instancia):
    tamanho = int(len(populacao)/2)
    for i in range(tamanho):
        paiA = populacao[random.randint(0, tamanho - 1)]['representacao']
        paiB = populacao[random.randint(0, tamanho - 1)]['representacao']
        filhoA = paiA.copy()
        filhoB = paiB.copy()
        filhoA[ponto_corte:] = paiB[ponto_corte:]
        filhoB[ponto_corte:] = paiA[ponto_corte:]
        populacao.append(criar_individuo(filhoA, instancia))
        populacao.append(criar_individuo(filhoB, instancia))
    
    return populacao


Função para aplicar a mutação em um grupo de indivíduos da população.

In [ ]:
def aplicar_mutacao(populacao, instancia):
    comprimento_cromossomo = instancia['numero_itens']
    individuos_mutantes = random.sample(range(len(populacao) - 1), MUTANTES)
    for individuo in individuos_mutantes:
        gene = random.randint(0, comprimento_cromossomo - 1)
        if populacao[individuo]['representacao'][gene] == 1: 
            populacao[individuo]['representacao'][gene] = 0
        else:
            populacao[individuo]['representacao'][gene] = 1
        populacao[individuo]['aptidao'] = grau_aptidao(populacao[individuo], instancia)
        
    return populacao


Função para gerar uma população aleatória de indivíduos

In [ ]:
def gerar_populacao_inicial_aleatoria(instancia, tamanho_populacao):
    populacao = []
    tamanho_cromossomo = instancia['numero_itens']
    for _ in range(tamanho_populacao):
        representacao_aleatoria = np.random.choice([0,1], tamanho_cromossomo, p=[0.9, 0.1])
        individuo = criar_individuo(representacao_aleatoria, instancia)
        populacao.append(individuo)
    
    return populacao


Função para calcular o grau de aptidão de um indivíduo

In [ ]:
def grau_aptidao(individuo, instancia):
    grau = 0
    for i, gene in enumerate(individuo['representacao']):
        grau = grau + (gene * instancia['itens'][i]['valor'])   

    return grau


Função para avaliar se a solução atual é melhor do que a melhor solução obtida até o momento. 

In [ ]:
def avaliar_solucao(solucao_otima, populacao, instancia):
    solucao_atual = populacao[0]['aptidao']
    viavel = solucao_viavel(populacao[0]['representacao'], instancia)
    if solucao_atual > solucao_otima and viavel:
        return solucao_atual
    
    return solucao_otima


Função para avaliar se uma solução é viável

In [ ]:
def solucao_viavel(individuo, instancia):
    peso_total = 0
    for i, gene in enumerate(individuo):
        peso_total = peso_total + (gene * instancia['itens'][i]['peso'])   
        if peso_total > instancia['capacidade']:
            return False

    return True


### Programa principal

Definição de parâmetros para o algoritmo genético. Você pode variar esses parâmetros para poder entender melhor o comportamento do algoritmo. 
A variável "NOME_INSTANCIA" permite que você selecione a instância que deseja ser avaliada pelo algoritmo. 

In [ ]:
TAMANHO_POPULACAO = 10
NUMERO_GERACOES = 10
TAXA_SELECAO = 0.5
TAXA_MUTACAO = 0.05
NOME_INSTANCIA = "f1_l-d_kp_10_269"

Constantes calculadas a partir dos parâmetros definidos anteriormente

In [ ]:
SELECIONADOS = int(TAMANHO_POPULACAO * TAXA_SELECAO)
MUTANTES = int(TAMANHO_POPULACAO * TAXA_MUTACAO)

Inicialização da variável "solucao_otima" com -1

In [ ]:
solucao_otima = -1

Leitura da instância do problema

In [ ]:
instancia = ler_instancia(NOME_INSTANCIA)
print(f"Capacidade máxima da mochila: {instancia['capacidade']}")
print(f"Número de itens disponíveis: {instancia['numero_itens']}")


Capacidade máxima da mochila: 269
Número de itens disponíveis: 10


Determinação do comprimento do cromossomo. Como estamos usando uma representação de bits, então cada posição do vetor é usada para representar se o item está ou não inserido na mochila. Portanto, o comprimento do cromossomo é exatamente igual a quantidade de itens disponíveis para serem colocados na mochila. 

In [ ]:
comprimento_cromossomo = instancia['numero_itens']


10

Gerando população inicial

In [ ]:
populacao = gerar_populacao_inicial_aleatoria(instancia, TAMANHO_POPULACAO)

Iniciando laço de repetição para evolução dos indivíduos. A quantidade de iterações desse laço é determinado pelo parâmetro "NUMERO_GERACOES" que foi definido anteriormente. 

In [28]:
for geracao in range(NUMERO_GERACOES):
  # Definição aleatória do ponto de corte para realizar a reprodução dos indivíduos.
  ponto_corte = random.randint(0, comprimento_cromossomo - 1)

  # Executa a selação dos indivíduos mais aptos.
  individuos_selecionados = selecionar_individuos(populacao)

  # Avalia se a melhor solução encontrada na geração atual é melhor do que a melhor solução encontrada até o momento. 
  solucao_otima = avaliar_solucao(solucao_otima, populacao, instancia)
  print(f"Geração {geracao+1} - Melhor solução: {solucao_otima}")

  # Executa a reprodução dos indivíduos.
  populacao = reproduzir(individuos_selecionados, ponto_corte, instancia)

  # Aplica uma mutação em alguns indivíduos da nova população. 
  populacao = aplicar_mutacao(populacao, instancia)


Geração 1 - Melhor solução: 116
Geração 2 - Melhor solução: 116
Geração 3 - Melhor solução: 116
Geração 4 - Melhor solução: 116
Geração 5 - Melhor solução: 116
Geração 6 - Melhor solução: 116
Geração 7 - Melhor solução: 116
Geração 8 - Melhor solução: 116
Geração 9 - Melhor solução: 116
Geração 10 - Melhor solução: 116
